In [1]:
from pyspark.sql.functions import col, when, to_date, lit

# 1. Configuração para evitar erro de datas antigas
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_sistemas@onelake.dfs.fabric.microsoft.com/lk_systemmax.Lakehouse/Tables/tab_funcionarios_dia"

df_resultado = spark.sql("""
SELECT
    HS.DATA, 
    HS.CODCOLIGADA, 
    HS.CHAPA,
    PP.NOME, 
    PP.DTNASCIMENTO, 
    HS.SITUACAO, 
    HS.FUNCAO, 
    HS.SECAO,
    HS.EQUIPE AS CODEQUIPE,
    PP.CPF,
    PP.DEFICIENTEFISICO,
    PP.SEXO 
FROM tab_gold_dim_funcionario_historico AS HS 
INNER JOIN ppessoa AS PP
ON HS.CODPESSOA = PP.CODIGO
""")

# 3. Corrigir datas inválidas (< 1900-01-01)
data_minima = to_date(lit("1900-01-01"))
data_padrao = to_date(lit("2000-01-01"))

df_corrigido = df_resultado \
    .withColumn("DTNASCIMENTO", when(col("DTNASCIMENTO") < data_minima, data_padrao).otherwise(col("DTNASCIMENTO")))


# Salva no Lakehouse de destino usando caminho físico
df_corrigido.write.format("delta").mode("overwrite").save(path_destino)

StatementMeta(, faf05c1e-5413-42ee-bf56-6cc231d7df9a, 3, Finished, Available, Finished)